In [3]:
import pandas as pd

In [4]:
train_file_path=r"C:\Users\HPr\Desktop\Projekte\Travel_Package_Project\package\ordner\data_ingestion\2022-07-28-22-24-08\ingested_data\train\TourismData.csv"
df=pd.read_csv(train_file_path)

In [5]:
df.head()

,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,0,NaN,Self Enquiry,3,6.0,Small Business,Male,2,1.0,Deluxe,5.0,Married,2.0,0,4,0,0.0,Manager,NaN
1,0,NaN,Self Enquiry,1,8.0,Salaried,Male,3,3.0,Basic,3.0,Married,2.0,0,5,0,2.0,Executive,18477.0
2,0,35.0,Company Invited,1,10.0,Salaried,Male,3,4.0,Deluxe,3.0,Unmarried,2.0,0,2,1,1.0,Manager,26434.0
3,0,38.0,Self Enquiry,1,26.0,Salaried,Male,4,4.0,Basic,4.0,Married,6.0,0,4,0,2.0,Executive,21700.0
4,0,38.0,Self Enquiry,1,6.0,Salaried,Female,2,3.0,Basic,5.0,Single,4.0,0,2,1,1.0,Executive,17619.0


In [6]:
df['ProdTaken']

0       0
1       0
2       0
3       0
4       0
       ..
3905    0
3906    0
3907    1
3908    1
3909    1
Name: ProdTaken, Length: 3910, dtype: int64

In [7]:
# creating list of category columns that are not object type
cat_cols = ["CityTier","ProdTaken","NumberOfPersonVisiting","NumberOfChildrenVisiting","PreferredPropertyStar","Passport","PitchSatisfactionScore","OwnCar"]
df[cat_cols] = df[cat_cols].astype("category")

In [8]:
# selecting all object datatypes and converting to category
cols = df.select_dtypes(["object"])
for i in cols.columns:
    df[i] = df[i].astype("category")

In [9]:
# check the dataset for updated datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3910 entries, 0 to 3909
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   ProdTaken                 3910 non-null   category
 1   Age                       3727 non-null   float64 
 2   TypeofContact             3893 non-null   category
 3   CityTier                  3910 non-null   category
 4   DurationOfPitch           3716 non-null   float64 
 5   Occupation                3910 non-null   category
 6   Gender                    3910 non-null   category
 7   NumberOfPersonVisiting    3910 non-null   category
 8   NumberOfFollowups         3871 non-null   float64 
 9   ProductPitched            3910 non-null   category
 10  PreferredPropertyStar     3891 non-null   category
 11  MaritalStatus             3910 non-null   category
 12  NumberOfTrips             3803 non-null   float64 
 13  Passport                  3910 non-null   catego

In [10]:
# check number of null records
df.isna().sum()

ProdTaken                     0
Age                         183
TypeofContact                17
CityTier                      0
DurationOfPitch             194
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            39
ProductPitched                0
PreferredPropertyStar        19
MaritalStatus                 0
NumberOfTrips               107
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     53
Designation                   0
MonthlyIncome               183
dtype: int64

In [11]:
# replace the missing values with median income w.r.t the customer"s designation
df["MonthlyIncome"] = df.groupby(["Designation"])["MonthlyIncome"].transform(lambda x: x.fillna(x.median()))
df["Age"] = df.groupby(["Designation"])["Age"].transform(lambda x: x.fillna(x.median()))

In [12]:
#Treat other numerical columns for missing values¶
# create list of numerical columns
import numpy as np
missing_numerical = df.select_dtypes(include=np.number).columns.tolist()

In [13]:
# remove Age and MonthlyIncome as we have already treated these columns
missing_numerical.remove("MonthlyIncome")
missing_numerical.remove("Age")

In [14]:
# function for replacing with the Median value of the attributes
medianFiller = lambda x: x.fillna(x.median()) 

In [15]:
# apply the function
df[missing_numerical] = df[missing_numerical].apply(medianFiller,axis=0)


In [16]:
#create a list of categorical columns
cat_cols =  df.select_dtypes(["category"])

#get the valuecounts
for i in cat_cols.columns:
    print(cat_cols[i].value_counts())
    print("-"*50)
    print("\n")

0    3174
1     736
Name: ProdTaken, dtype: int64
--------------------------------------------------


Self Enquiry       2751
Company Invited    1142
Name: TypeofContact, dtype: int64
--------------------------------------------------


1    2557
3    1188
2     165
Name: CityTier, dtype: int64
--------------------------------------------------


Salaried          1892
Small Business    1660
Large Business     356
Free Lancer          2
Name: Occupation, dtype: int64
--------------------------------------------------


Male       2321
Female     1471
Fe Male     118
Name: Gender, dtype: int64
--------------------------------------------------


3    1928
2    1131
4     821
1      28
5       2
Name: NumberOfPersonVisiting, dtype: int64
--------------------------------------------------


Basic           1474
Deluxe          1378
Standard         601
Super Deluxe     278
King             179
Name: ProductPitched, dtype: int64
--------------------------------------------------


3.0    

In [17]:
#Treat the other columns for missing values
# treating missing values in remaining categorical variables
df["TypeofContact"] = df["TypeofContact"].fillna("Self Enquiry")
df["NumberOfChildrenVisiting"] = df["NumberOfChildrenVisiting"].fillna(1.0)
df["PreferredPropertyStar"] = df["PreferredPropertyStar"].fillna(3.0)

In [18]:
#Treat the error in Gender column
# treating error
df.Gender = df.Gender.replace("Fe Male","Female")

# verify the update
df.Gender.value_counts()

Male      2321
Female    1589
Name: Gender, dtype: int64

In [19]:
#Verify the missing value treatment
# count of null records
df.isna().sum()

ProdTaken                   0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch             0
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
MaritalStatus               0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
OwnCar                      0
NumberOfChildrenVisiting    0
Designation                 0
MonthlyIncome               0
dtype: int64

In [20]:
# summary of numerical columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,3910.0,37.467263,9.169467,18.0,31.00,36.0,43.0,61.0
DurationOfPitch,3910.0,15.422251,8.437237,5.0,9.00,13.0,19.0,127.0
NumberOfFollowups,3910.0,3.707928,0.999930,1.0,3.00,4.0,4.0,6.0
NumberOfTrips,3910.0,3.228900,1.824630,1.0,2.00,3.0,4.0,22.0
MonthlyIncome,3910.0,23549.456266,5307.094328,4678.0,20475.25,22372.5,25433.0,98678.0


In [21]:
# summary of categorical columns
df.describe(include="category").T

,count,unique,top,freq
ProdTaken,3910,2,0,3174
TypeofContact,3910,2,Self Enquiry,2768
CityTier,3910,3,1,2557
Occupation,3910,4,Salaried,1892
Gender,3910,2,Male,2321
NumberOfPersonVisiting,3910,5,3,1928
ProductPitched,3910,5,Basic,1474
PreferredPropertyStar,3910.0,3.0,3.0,2403.0
MaritalStatus,3910,4,Married,1874
Passport,3910,2,0,2765


In [22]:
#Lets check the percentage of Outliers with IQR
outlier_num = df.select_dtypes(include=np.number)

In [23]:
outlier_num

,Age,DurationOfPitch,NumberOfFollowups,NumberOfTrips,MonthlyIncome
0,36.0,6.0,1.0,2.0,22876.0
1,32.0,8.0,3.0,2.0,18477.0
2,35.0,10.0,4.0,2.0,26434.0
3,38.0,26.0,4.0,6.0,21700.0
4,38.0,6.0,3.0,4.0,17619.0
...,...,...,...,...,...
3905,39.0,21.0,3.0,1.0,28341.0
3906,48.0,6.0,4.0,1.0,30402.0
3907,25.0,10.0,4.0,2.0,23255.0
3908,26.0,10.0,4.0,7.0,22709.0


In [24]:
df.isna().sum()

ProdTaken                   0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch             0
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
MaritalStatus               0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
OwnCar                      0
NumberOfChildrenVisiting    0
Designation                 0
MonthlyIncome               0
dtype: int64

In [25]:
# find the 25th percentile and 75th percentile.
Q1 = df.quantile(0.25)            
Q3 = df.quantile(0.75)

# Inter Quantile Range (75th percentile - 25th percentile)
IQR = Q3 - Q1                           

# find lower and upper bounds for all values. All values outside these bounds are outliers
lower=Q1-1.5*IQR                        
upper=Q3+1.5*IQR

((outlier_num<lower)|(outlier_num>upper)).sum()/len(df)*100

Age                  0.000000
DurationOfPitch      2.352941
NumberOfFollowups    6.470588
NumberOfTrips        2.276215
MonthlyIncome        7.391304
dtype: float64

In [26]:
# split the columns in Dependent and Independent Set
X= df.drop(["ProdTaken","PitchSatisfactionScore","ProductPitched","NumberOfFollowups","DurationOfPitch"],axis=1)
y= df["ProdTaken"]

In [27]:
# use get_dummies function to convert the categorical columns
X = pd.get_dummies(X, drop_first=True)

In [28]:
# splitting data into training and test set, use stratify to maintain the original distribution of Dependent variable as of original set
from sklearn.model_selection import GridSearchCV, train_test_split

X_train,X_test, y_train, y_test =train_test_split(X,y, test_size=0.3, random_state=25,stratify=y)

# creating a list of column names
feature_names = X_train.columns.to_list()
feature_names

['Age',
 'NumberOfTrips',
 'MonthlyIncome',
 'TypeofContact_Self Enquiry',
 'CityTier_2',
 'CityTier_3',
 'Occupation_Large Business',
 'Occupation_Salaried',
 'Occupation_Small Business',
 'Gender_Male',
 'NumberOfPersonVisiting_2',
 'NumberOfPersonVisiting_3',
 'NumberOfPersonVisiting_4',
 'NumberOfPersonVisiting_5',
 'PreferredPropertyStar_4.0',
 'PreferredPropertyStar_5.0',
 'MaritalStatus_Married',
 'MaritalStatus_Single',
 'MaritalStatus_Unmarried',
 'Passport_1',
 'OwnCar_1',
 'NumberOfChildrenVisiting_1.0',
 'NumberOfChildrenVisiting_2.0',
 'NumberOfChildrenVisiting_3.0',
 'Designation_Executive',
 'Designation_Manager',
 'Designation_Senior Manager',
 'Designation_VP']

In [29]:
from sklearn import metrics
# libraries to build Linear Regression Model
from sklearn.linear_model import LogisticRegression

# libraries to build decision tree classifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

# libraries to build ensemble models
from sklearn.ensemble import (
    BaggingClassifier,
    RandomForestClassifier,
    AdaBoostClassifier, 
    GradientBoostingClassifier,
    StackingClassifier,
)

# library for stacking classifier
from sklearn.ensemble import StackingClassifier

# to tune different models
from sklearn.model_selection import GridSearchCV, train_test_split

# to get diferent metric scores
from sklearn.metrics import (
    f1_score,accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    plot_confusion_matrix,
    make_scorer,
    roc_auc_score,
    precision_recall_curve,
    roc_curve,
    classification_report,
)

In [30]:
#Check the split of target variable ProdTaken
print("Original ProdTaken True Values    : {0} ({1:0.2f}%)".format(len(df.loc[df["ProdTaken"] == 1]), (len(df.loc[df["ProdTaken"] == 1])/len(df.index)) * 100))
print("Original ProdTaken False Values   : {0} ({1:0.2f}%)".format(len(df.loc[df["ProdTaken"] == 0]), (len(df.loc[df["ProdTaken"] == 0])/len(df.index)) * 100))
print("")
print("Training ProdTaken True Values    : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 1]), (len(y_train[y_train[:] == 1])/len(y_train)) * 100))
print("Training ProdTaken False Values   : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 0]), (len(y_train[y_train[:] == 0])/len(y_train)) * 100))
print("")
print("Test ProdTaken True Values        : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 1]), (len(y_test[y_test[:] == 1])/len(y_test)) * 100))
print("Test ProdTaken False Values       : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 0]), (len(y_test[y_test[:] == 0])/len(y_test)) * 100))

Original ProdTaken True Values    : 736 (18.82%)
Original ProdTaken False Values   : 3174 (81.18%)

Training ProdTaken True Values    : 515 (18.82%)
Training ProdTaken False Values   : 2222 (81.18%)

Test ProdTaken True Values        : 221 (18.84%)
Test ProdTaken False Values       : 952 (81.16%)


#### Descision Tree Classifier

In [31]:
# DecistionTreeClassifier with gini and class_weight for appropriate importance
dtc = DecisionTreeClassifier(criterion="gini",class_weight={0:0.15,1:0.85},random_state=1)

In [32]:
# fit the model on training dataset
dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight={0: 0.15, 1: 0.85}, random_state=1)

In [33]:
dtc.score(X_train, y_train)

1.0

In [34]:
dtc.score(X_test, y_test)

0.8516624040920716

In [35]:
dtc.predict(X_test)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [36]:
y_pred=dtc.predict(X_test)

In [37]:
# to get diferent metric scores
from sklearn.metrics import (
    f1_score,accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    plot_confusion_matrix,
    make_scorer,
    roc_auc_score,
    precision_recall_curve,
    roc_curve,
    classification_report,
)

In [38]:
f1_score(y_test,y_pred)

0.5972222222222222

In [39]:
confusion_matrix(y_pred,y_test)

array([[870,  92],
       [ 82, 129]], dtype=int64)

In [40]:
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : \n', ClassificationReport )

Classification Report is : 
               precision    recall  f1-score   support

           0       0.90      0.91      0.91       952
           1       0.61      0.58      0.60       221

    accuracy                           0.85      1173
   macro avg       0.76      0.75      0.75      1173
weighted avg       0.85      0.85      0.85      1173



#### Hypertuned Decision Tree Classifier

In [41]:
# choose the type of classifier. 
dtc1 = DecisionTreeClassifier(random_state=1,class_weight = {0:.15,1:.85}) #adding classweights 

# defining the Hyperparameters
parameters = {"max_depth": np.arange(10,60,10), 
            "criterion": ["gini","entropy"],
            "min_samples_leaf": [ 2, 5, 7, 10],
            "max_leaf_nodes" : [3, 5, 10,15],}
# type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# run the grid search with the above parameters
grid_obj = GridSearchCV(dtc1, parameters, scoring=scorer,cv=5)
grid_obj = grid_obj.fit(X_train, y_train)

# set to the best combination of parameters
dtcht = grid_obj.best_estimator_

# fit the best algorithm to the data. 
dtcht.fit(X_train, y_train)

DecisionTreeClassifier(class_weight={0: 0.15, 1: 0.85}, criterion='entropy',
                       max_depth=10, max_leaf_nodes=10, min_samples_leaf=2,
                       random_state=1)

In [42]:
dtcht.score(X_train, y_train)

0.731457800511509

In [43]:
dtcht.score(X_test, y_test)

0.7203751065643649

In [44]:
y_pred=dtcht.predict(X_test)

In [45]:
f1_score(y_test,y_pred)

0.47096774193548385

In [46]:
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : \n', ClassificationReport )

Classification Report is : 
               precision    recall  f1-score   support

           0       0.90      0.73      0.81       952
           1       0.37      0.66      0.47       221

    accuracy                           0.72      1173
   macro avg       0.63      0.70      0.64      1173
weighted avg       0.80      0.72      0.75      1173



In [47]:
# baggingClassifier
bagging = BaggingClassifier(random_state=44)

# fit the model on training dataset
bagging.fit(X_train, y_train)

BaggingClassifier(random_state=44)

In [48]:
bagging.score(X_train, y_train)

0.9886737303617099

In [49]:
bagging.score(X_test, y_test)

0.8525149190110827

In [50]:
y_pred=bagging.predict(X_test)

In [51]:
f1_score(y_test,y_pred)

0.5286103542234332

In [52]:
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : ', ClassificationReport )

Classification Report is :                precision    recall  f1-score   support

           0       0.88      0.95      0.91       952
           1       0.66      0.44      0.53       221

    accuracy                           0.85      1173
   macro avg       0.77      0.69      0.72      1173
weighted avg       0.84      0.85      0.84      1173



In [53]:
confusion_matrix(y_pred,y_test)

array([[903, 124],
       [ 49,  97]], dtype=int64)

#### obervation
The model is over fitting. The Bagging classifier has a better accuracy metric and the F1 score is also higher. But model only predicts 10.97% of the total 13% of True positives.

#### Hypertuned Bagging Classifier

In [54]:
# set the parameters
parameters = {
              "n_estimators":np.arange(10,60,10),
              "max_features": [0.7,0.8,0.9],
              "max_samples": [0.7,0.8,0.9], 
             }

# assigning Bootstrap = True to select features with Replacement
hyperbagging = BaggingClassifier(random_state=1,bootstrap=True)

# type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# run the grid search
grid_obj = GridSearchCV(hyperbagging, parameters, scoring=scorer,cv=5)
grid_obj = grid_obj.fit(X_train, y_train)

# set the clf to the best combination of parameters
bgcht = grid_obj.best_estimator_

# fit the best algorithm to the data.
bgcht.fit(X_train,y_train)

BaggingClassifier(max_features=0.9, max_samples=0.9, n_estimators=50,
                  random_state=1)

In [55]:
bgcht.score(X_test,y_test)

0.8772378516624041

In [56]:
y_pred=bagging.predict(X_test)

In [57]:
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : \n', ClassificationReport )

Classification Report is : 
               precision    recall  f1-score   support

           0       0.88      0.95      0.91       952
           1       0.66      0.44      0.53       221

    accuracy                           0.85      1173
   macro avg       0.77      0.69      0.72      1173
weighted avg       0.84      0.85      0.84      1173



In [58]:
confusion_matrix(y_pred,y_test)

array([[903, 124],
       [ 49,  97]], dtype=int64)

In [59]:
roc_auc_score(y_test,y_pred, average='micro')

0.6937217194570136

#### Observations:
We see that the Train and Test Accuracy and F1Score Performance has increased after tuning compared to the previous models.
 The Model is over-fitting as the difference between Train and Test scores are very high. The Model seems to identify all non-buyers better as the False Positive value is low.

#### ADABOOST CLASSIFIER

In [60]:
# adaboost classifier
ada = AdaBoostClassifier(random_state=1)

# fit the model on training set
ada.fit(X_train,y_train)

AdaBoostClassifier(random_state=1)

In [61]:
ada.score(X_train,y_train)

0.8542199488491049

In [62]:
ada.score(X_test,y_test)

0.8337595907928389

In [63]:
y_pred=ada.predict(X_test)

In [64]:
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : ', ClassificationReport )

Classification Report is :                precision    recall  f1-score   support

           0       0.85      0.97      0.90       952
           1       0.64      0.26      0.37       221

    accuracy                           0.83      1173
   macro avg       0.75      0.61      0.64      1173
weighted avg       0.81      0.83      0.80      1173



In [65]:
confusion_matrix(y_pred,y_test)

array([[920, 163],
       [ 32,  58]], dtype=int64)

#### observation
The metrics for ADA boost model is close and comparable for train and test set

#### Hypertuned Random Forest Classifier

In [66]:
rfc1 = RandomForestClassifier(class_weight={0:0.15,1:0.85},random_state=1)

# grid of parameters to choose from
parameters = {"n_estimators": np.arange(110,251,501),
              "min_samples_leaf": np.arange(1,6,1),
              "max_features":["log2",0.7,0.9,"auto"],
              "max_samples": np.arange(0.3, 0.7, None),
             }

# type of scoring used to compare parameter combinations
scorer = metrics.make_scorer(metrics.f1_score)

# run the grid search
grid_obj = GridSearchCV(rfc1, parameters, scoring=scorer,cv=5)
grid_obj = grid_obj.fit(X_train, y_train)

# set the clf to the best combination of parameters
rfcht = grid_obj.best_estimator_

# fit the best algorithm to the data.
rfcht.fit(X_train, y_train)

c:\Users\HPr\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\HPr\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
c:\Users\HPr\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for Random

RandomForestClassifier(class_weight={0: 0.15, 1: 0.85}, max_features=0.9,
                       max_samples=0.3, min_samples_leaf=5, n_estimators=110,
                       random_state=1)

In [67]:
rfc1.fit(X_train,y_train)

RandomForestClassifier(class_weight={0: 0.15, 1: 0.85}, random_state=1)

In [68]:
rfc1.score(X_train,y_train)

1.0

In [69]:
rfc1.score(X_test,y_test)

0.8687127024722933

In [70]:
y_pred=rfc1.predict(X_test)

In [71]:
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : \n ', ClassificationReport )

Classification Report is : 
                precision    recall  f1-score   support

           0       0.87      0.98      0.92       952
           1       0.84      0.38      0.52       221

    accuracy                           0.87      1173
   macro avg       0.85      0.68      0.72      1173
weighted avg       0.87      0.87      0.85      1173



In [72]:
confusion_matrix(y_test,y_pred)

array([[936,  16],
       [138,  83]], dtype=int64)

In [73]:
# library for stacking classifier
from sklearn.ensemble import StackingClassifier

#### Stacking Classifier 

In [74]:
# define the estimators for the model
estimators = [("Random Forest",rfcht),("ADA Boosting",ada),("Decision Tree",dtc)]

# define the final estimator
final_estimator = rfcht

# choose the classifier
sc= StackingClassifier(estimators=estimators,final_estimator=final_estimator)

# train the model on training set
sc.fit(X_train,y_train)

StackingClassifier(estimators=[('Random Forest',
                                RandomForestClassifier(class_weight={0: 0.15,
                                                                     1: 0.85},
                                                       max_features=0.9,
                                                       max_samples=0.3,
                                                       min_samples_leaf=5,
                                                       n_estimators=110,
                                                       random_state=1)),
                               ('ADA Boosting',
                                AdaBoostClassifier(random_state=1)),
                               ('Decision Tree',
                                DecisionTreeClassifier(class_weight={0: 0.15,
                                                                     1: 0.85},
                                                       random_state=1))],
                   final_estimator=RandomForestClassifier(class_weight={0: 0.15,
                                                                        1: 0.85},
                                                          max_features=0.9,
                                                          max_samples=0.3,
                                                          min_samples_leaf=5,
                                                          n_estimators=110,
                                                          random_state=1))

In [75]:
sc.score(X_test,y_test)

0.8115942028985508

In [76]:
y_pred=sc.predict(X_test)

In [77]:
confusion_matrix(y_test,y_pred)

array([[806, 146],
       [ 75, 146]], dtype=int64)

In [78]:
ClassificationReport = classification_report(y_test,y_pred)
print('Classification Report is : \n', ClassificationReport )

Classification Report is : 
               precision    recall  f1-score   support

           0       0.91      0.85      0.88       952
           1       0.50      0.66      0.57       221

    accuracy                           0.81      1173
   macro avg       0.71      0.75      0.72      1173
weighted avg       0.84      0.81      0.82      1173



In [79]:
models =  ['Desicion Tree Classifier','Hypertuned Descision Tree Classifier','Bagging Classifier','Hypertuned Bagging Classifier','Ada Boost','Hypertuned Random Forest','Stacking Classifier']
Score= [0.8516,0.7203,0.8525,0.8722,0.8337,0.8687,0.8115]

dict(zip(models,Score))

{'Desicion Tree Classifier': 0.8516,
 'Hypertuned Descision Tree Classifier': 0.7203,
 'Bagging Classifier': 0.8525,
 'Hypertuned Bagging Classifier': 0.8722,
 'Ada Boost': 0.8337,
 'Hypertuned Random Forest': 0.8687,
 'Stacking Classifier': 0.8115}

#### conclusion

Bagging with Decision Tree has the highest F1 Score but it is overfitting the data.

Despite having lower F1 Score, the Hypertuned Random Forest has more generalized metrics and does not seem to be over-fitting the data. Making it the best suited for future analysis.

Most of the models show comparable scores between training and testing sets.

In [80]:
import dill
model_pkl = r'C:\Users\HPr\Desktop\Projekte\Travel_Package_Project\package\ordner\model_trainer\2022-08-01-14-33-46\trained_model\model.yaml\model.yaml'
dill.dump(rfc1,open(model_pkl,'wb'))

In [ ]:
saved_Bestmodel

RandomForestClassifier(class_weight={0: 0.15, 1: 0.85}, random_state=1)

In [84]:
import dill
model_pkl_file= r'C:\Users\HPr\Desktop\Projekte\Travel_Package_Project\trained_model_dir\model_pkl'
dill.dump(rfc1,open(model_pkl_file,'wb'))
saved_Bestmodel = dill.load(open(model_pkl_file,'rb'))